In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import utility as util

In [ ]:
help(util)

In [ ]:
path = '/kaggle/input/kaggle-survey-2020/'
filename = 'kaggle_survey_2020_responses.csv'
data = util.load_data(path, filename)

In [ ]:
data.head()

Devide questions and answers

In [ ]:
questions = data.iloc[0,:]

In [ ]:
answers = data.iloc[1:,:]

The first question is about age.

In [ ]:
ages = answers['Q1'].value_counts()

In [ ]:
ax = ages.plot.bar(x='lab', y='val', rot=0)

The first question is about gender.

In [ ]:
gender = answers['Q2'].value_counts()

In [ ]:
ax1 = gender.plot.pie()

The third question is about the country.

In [ ]:
answers["Q3"]

In [ ]:
answers['Q3'].value_counts()

In [ ]:
answers['Q3'].replace("United States of America", "United States", inplace=True)
answers['Q3'].replace("South Korea", "Korea, Republic of", inplace=True)
answers['Q3'].replace("Republic of Korea", "Korea, Republic of", inplace=True)
answers['Q3'].replace("Korea", "Korea, Republic of", inplace=True)
answers['Q3'].replace("Russia", "Russian Federation", inplace=True)
answers['Q3'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom", inplace=True)
answers['Q3'].replace("Iran, Islamic Republic of...", "Iran, Islamic Republic of", inplace=True)
answers['Q3'].replace("Taiwan", "Taiwan, Province of China", inplace=True)

In [ ]:
# generate country code  based on country name 
import pycountry 
def alpha2code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_2
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE

In [ ]:
# create a column for code 
answers['CODE']=alpha2code(answers["Q3"])
answers.head()

In [ ]:
countries = answers['CODE'].value_counts()

In [ ]:
countries = pd.DataFrame(countries)

In [ ]:
df = countries.reset_index()

In [ ]:
df = df.rename(columns={"CODE": "numbers", "index": "CODE"})

In [ ]:
#installation
# !pip install geopy
#function to get longitude and latitude data from country name
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="tomo.krmr@gmail.com")
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return loc.latitude, loc.longitude
    except:
        # Return missing value
        return 0, 0

In [ ]:
df['LOC'] = df['CODE'].apply(geolocate)

In [ ]:

# # plot confirmed cases world map 
# df.plot(column='numbers', scheme="quantiles",
#            figsize=(25, 20),
#            legend=True,cmap='coolwarm')
# plt.title('kaggle survey participants distribution in the world',fontsize=25)
# # add countries names and numbers 
# # for i in range(0,10):
# #     plt.text(float(df.longitude[i]),float(merge.latitude[i]),"{}\n{}".format(merge.name[i],merge.Confirmed_Cases[i]),size=10)
# plt.show()

In [ ]:
#installation
# !pip install folium
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)

In [ ]:
#for each coordinate, create circlemarker of user percent
for i in range(len(df)):
        loc = df.iloc[i]['LOC']
        radius=5
        popup_text = """Country : {}<br>
                    %of Users : {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['CODE'],
                                   df.iloc[i]['numbers']
                                   )
        folium.CircleMarker(location = loc, radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

## Thanks for
[data-visualization-how-to-plot-a-map-with-geopandas-in-python](https://melaniesoek0120.medium.com/data-visualization-how-to-plot-a-map-with-geopandas-in-python-73b10dcd4b4b)

[using-python-to-create-a-world-map-from-a-list-of-country-names](https://towardsdatascience.com/using-python-to-create-a-world-map-from-a-list-of-country-names-cd7480d03b10)